In [1]:
import sqlite3
import numpy as np
import pandas as pd
import json
import math
import random
from IPython.display import clear_output

cnx = sqlite3.connect('data/ultimate_player_database.db')

In [40]:
query = "SELECT game_data FROM sets WHERE game_data != '[]'"

df = pd.read_sql_query(query, cnx)

data = list()

for k in range(len(df)):

    if k % 50000 == 0:
        clear_output()
        print("{prop} % complete".format(prop = math.floor(100*k/len(df))))

    str = df.loc[k][0]

    while True:
        id1 = str.find("{")
        id2 = str.find("}")
        if (id1 != -1 ) and (id2 != -1):
            new_data = json.loads(str[id1:id2+1])
            data.append(new_data)
        else:
            break  
        str = str[id2+1:]

game_df = pd.DataFrame(data).fillna(-1)

game_df.to_csv("data/game_data.csv",index = False)

clear_output()
print("100 % complete")


100 % complete


In [2]:
game_df = pd.read_csv("data/game_data.csv",dtype={'winner_id' : "string", 'loser_id' : "string", 'winner_score' : "string", 'loser_score' : "string","winner_char" : "string","loser_char" : "string"})
game_df = game_df.drop("winner_score",axis=1).drop("loser_score",axis=1)
game_df["winner_char"] = game_df.apply(lambda x: x["winner_char"].replace("ultimate/",""),axis=1)
game_df["loser_char"] = game_df.apply(lambda x: x["loser_char"].replace("ultimate/",""),axis=1)

We need to transform our data to be impartial to the winner or loser. We now have `winner_id`, `loser_id`, `winner_char`,`loser_char`, ... ect but what we want is `p_1_id`,`p2_id`, `p1_char`,`p2_char`, `p1_won`. Otherwise, we wouldn't be predicting anything because we would know who won beforehand.

In [3]:
def condswap(b,tup):
    # conditional swap
    if b: 
        return (tup[1],tup[0])
    else:
        return tup

def row_transform(row):
    output = dict()
    p1w = bool(random.getrandbits(1))
    output["p1_id"], output["p2_id"] = condswap(not p1w,(row["winner_id"],row["loser_id"]))
    output["p1_char"], output["p2_char"] = condswap(not p1w,(row["winner_char"],row["loser_char"]))
    output["stage"] = row["stage"]
    output["p1_won"] = p1w
    return output

random.seed(1984)

game_list = game_df.apply(row_transform,axis=1)
imp_game_df = pd.DataFrame.from_records(game_list)
imp_game_df

,p1_id,p2_id,p1_char,p2_char,stage,p1_won
0,Sunbaby,543328,mario,joker,Battlefield,False
1,543328,Sunbaby,joker,drmario,Yoshi's Story,True
2,1472816,1075251,pikachu,iceclimbers,Pokémon Stadium 2,False
3,1075251,1472816,iceclimbers,pikachu,Battlefield,True
4,challonge__MrRiceman,challonge__Loconotcoco,joker,wario,Pokémon Stadium 2,False
...,...,...,...,...,...,...
5927714,1263104,53481,lucina,palutena,Smashville,True
5927715,30044,230918,palutena,bowser,Pokémon Stadium 2,False
5927716,230918,30044,bowser,palutena,Town & City,False
5927717,230918,30044,bowser,palutena,Kalos Pokémon League,True


Something we should expect from our chosen model is player symmetry. Swapping the player 1 information with the player 2 information should give exactly the opposite yeild. Whatever our model is, we need to make sure that this is preserved.

`My dataset for the 131 final project is about win/loss rates per character in Smash Ultimate. The dataset has some information about the winners and losers of each game, but it doesn't have any metric for the skill level of each of the players, which I think would be an important predictor for the games. What I want to do is create a skill metric based on the set data.`

`I initially wanted to make this metric by finding the stationary distribution of a Markov Chain, where a player has a link to another player if they win a game against them. This would be similar to how Google Pagerank works. Unfortunately there are so many players ( approx. 150,000 ) that my computer can't find the stationary distribution of this graph. Maybe I could approximate the stationary distribution, but I wouldn't know how. I was thinking of making a more simple metric based on how many games the player has played, and what proportion of games they've won.`

`The problem with both of these metrics is that there might be data leakage from the test set to the training set if I compute it before...`

I want to add a column that gives information about how many games a player has played, and how many they have won so we can have a sort of "skill" metric. However, we need to be careful about how when we do this transformation to our data. If we do it before we train / test split, then, for example, the model could deduce that someone with 0 games won has a 0% chance of winning and get 100% accuracy for those values. Someone with 0 games previously won might have a *low* chance of winning, but it is certainly not zero. So, I will apply this transformation **after** splitting my data into training and testing sets.

Also, if we do k-fold cross validation, we may want to apply the transformation after folding as well. This is not as serious as leakage into the testing data but should improve model performance.

Another way to get around this would be to split the ENTIRE dataset in half, or maybe just keep 1 million entries, then calculate player ratio and skill from that. Then it would be independent of both the training and test, and everything would be MCAR.

In [4]:
def transform_games_played(dataframe):

    p1s = dict(dataframe["p1_id"].value_counts())
    p1wins = dict(dataframe[dataframe["p1_won"] == True]["p1_id"].value_counts())

    p2s = dict(dataframe["p2_id"].value_counts())
    p2wins = dict(dataframe[dataframe["p1_won"] == False]["p2_id"].value_counts())

    players = set(p1s.keys()).union(set(p2s.keys()))

    games_played = list()

    for item in players:
        totalsum = 0
        winsum = 0
        if item in p1s.keys():
            totalsum += p1s[item]
        if item in p2s.keys():
            totalsum += p2s[item]
        if item in p1wins.keys():
            winsum += p1wins[item]
        if item in p2wins.keys():
            winsum += p2wins[item]
        games_played.append({"player_id" : item, "games_played" : totalsum, "games_won" : winsum})

    games_played_df = pd.DataFrame(games_played)
    return games_played_df

def clean_games(given_df,skill_df):
    games_played_df = transform_games_played(skill_df)

    output_df = pd.merge(given_df,games_played_df, left_on = "p1_id", right_on = "player_id", how = "left")
    output_df.rename(columns = {"games_played" : "p1_games_played", "games_won" : "p1_games_won"}, inplace=True)
    output_df = pd.merge(output_df,games_played_df, left_on = "p2_id", right_on = "player_id", how = "left")
    output_df.rename(columns = {"games_played" : "p2_games_played", "games_won" : "p2_games_won"}, inplace=True)

    output_df["p1_games_won"].fillna(0,inplace=True)
    output_df["p1_games_played"].fillna(0,inplace=True)
    output_df["p2_games_won"].fillna(0,inplace=True)
    output_df["p2_games_played"].fillna(0,inplace=True)

    output_df.reset_index(inplace=True)
    output_df.drop(columns=["index","player_id_x","player_id_y"],inplace=True)
    return output_df


In [5]:
_skill_df = imp_game_df.sample(frac = 0.5, random_state=2049)
remaining_df = imp_game_df.drop(_skill_df.index)

clean_game_df = clean_games(remaining_df,_skill_df) # The output rows are independent of the games played and won.
skill_clean_game_df = clean_games(_skill_df,_skill_df) # The output rows are NOT independent of the games played and won.



In [6]:
clean_game_df.to_csv("data/clean_game_data.csv",index=False) # To be used for training/test.
skill_clean_game_df.to_csv("data/dependent_skill_data.csv",index = False) # To be used in case we want to create more metrics.